In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ast

#change dir as desired
os.chdir(r"C:\Users\ckunt\OneDrive\Documents\Masters work\HEC\18. Eleven Strategy\eleven-strategy\data")

csv_files = ['mutations_d75_train_localized.csv',
             'mutations_d77_train_localized.csv',
             'mutations_d78_train_localized.csv',
             'mutations_d91_train_localized.csv',
             'mutations_d92_train_localized.csv',
             'mutations_d93_train_localized.csv',
             'mutations_d94_train_localized.csv',
             'mutations_d95_train_localized.csv',
             ]

def combine_clean_files(csv_files: list[str],
                        explod: list[str] = 'l_codinsee') -> pd.DataFrame:
    """Combine csv files
    
    Args:
        csv_files: list of csv file names
        
    Returns:
        df: dataframe with combined dataframes
    """
    
    print('Getting dfs...')
    list_dfs = [pd.read_csv(f) for f in csv_files]
    
    print('Combining dfs...')
    df = pd.concat(list_dfs, ignore_index=True)
    
    # clean list columns
    print('Cleaning dfs')
    l_cols = [col for col in df.columns if col.startswith('l_')]
    df[l_cols] = df[l_cols].applymap(ast.literal_eval)
    df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
    
    print('Exploding columns...')
    return df.explode(explod)


In [4]:
df = combine_clean_files(csv_files)


Getting dfs...
Combining dfs...
Cleaning dfs
Exploding columns...


In [5]:
import geopandas as gpd
from shapely.geometry import Point

idf_reg = gpd.read_file('communes-dile-de-france-au-01-janvier.shp')
idf_reg['nomcom'] = idf_reg['nomcom'].str.encode('ISO-8859-1').str.decode('utf-8')
def convert_to_point(row):
    return Point(row["longitude"], row["latitude"])

df["lat_long"] = df.apply(convert_to_point, axis=1)

c:\Users\ckunt\anaconda3\envs\tf\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [6]:
df_points = gpd.GeoDataFrame(df, geometry="lat_long")
df_points.crs = {'init': 'epsg:4326'}
idf_reg = gpd.GeoDataFrame(idf_reg, geometry='geometry')
df_points = df_points.to_crs(idf_reg.crs)

c:\Users\ckunt\anaconda3\envs\tf\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
import folium

result = gpd.sjoin(idf_reg, df_points, how='inner', predicate='intersects')
grouped = result.groupby('nomcom').median().reset_index()
result = pd.merge(grouped[['nomcom', 'valeurfonc']], idf_reg[['nomcom', 'geometry']], how='left', on='nomcom')
dept_geo = gpd.GeoDataFrame(idf_reg.set_index('numdep')['geometry'], geometry='geometry').dissolve(by='numdep', aggfunc='sum')

In [8]:
import branca.colormap as cm

m = folium.Map(location=[48.856614, 2.3522219],
                    zoom_start = 10, tiles='cartodbpositron')

colormap_dept = cm.StepColormap(
    colors=['#00ae53', '#86dc76', '#daf8aa',
            '#ffe6a4', '#ff9a61', '#ee0028'],
    vmin=min(result['valeurfonc']),
    vmax=max(result['valeurfonc']),
    index=np.linspace(result['valeurfonc'].min(), result['valeurfonc'].max(), num=7))

In [9]:
style_function = lambda x: {
    'fillColor': colormap_dept(x['properties']['valeurfonc']),
    'color': '',
    'weight': 0.0001,
    'fillOpacity': 0.6
}

In [10]:
folium.GeoJson(
    dept_geo,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2.5,
        'fillOpacity': 0
    },
    name='Departement').add_to(m)

folium.GeoJson(
    gpd.GeoDataFrame(result),
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['nomcom', 'valeurfonc'],
        aliases=['Commune/Arrondisement', 'Valeur fonciere'],
        localize=False
    ),
    name='Community').add_to(m)

In [11]:
m